In [2]:
!pip install pygame

In [2]:
import math
import random
import pygame

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
import tkinter as tk

In [4]:
from tkinter import messagebox

In [5]:
class cube(object):
    rows = 0
    w = 0
def snake(object):
    body = []
    turns = {}
    def _init_(self, color, pos):
        self.color = color
        self.head = cube(pos)
        self.body.append(self.head)
        self.dirnx = 0
        self.dirny = 1 #direction for x and y snake movement
    def move(self):
        for events in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            keys = pygame.key.get_pressed() #gets all the keys values and if they were pressed or not, so if more than one keys are pressed it will adjust
            for key in keys:
                if keys[pygame.K_LEFT]:
                    
                if keys[pygame.K_RIGHT]:
                    
                if keys[pygame.K_UP]:
                    
                if keys[pygame.K_DOWN]:
                    
             
                    
            
            
        
    pass

def drawGrid(w, rows, surface): #to draw the boxes or to figure out where to draw the lines
    sizeBtwn = w // rows
    x = 0
    y = 0
    for l in range(rows):
        x = x + sizeBtwn
        y = y + sizeBtwn
        pygame.draw.line(surface, (255, 255, 255), (x, 0), (x, w)) #drawing lines, white color, position
        pygame.draw.line(surface, (255, 255, 255), (0, y), (w, y)) #drawing lines, white color, position
    
    

def redrawWindow(surface):
    global width, rows
    surface.fill((0,0,0)) #to fill the screen with black
    drawgrid(width, rows, surface)
    pygame.display.update()
    
    
def main():
    global width, rows
    width = 500
    rows = 20 #if 10 it will go much faster
    clock = pygame.time.Clock()
    win = pygame.display.set_mode((width, width))
    s = snake((255, 0, 0), (10, 10)) #for red snake and for the snake to start in the middle
    Flag = True
    while Flag:
        pygame.time.delay(50) #delay it by 50 ms, so that the program doesnt run too fast
        clock.tick(10) #for game to not run within 10 frames per sec (snake moves 10 blocks per sec)
        redrawWindow(win)       
    pass
main()

TypeError: snake() takes 1 positional argument but 2 were given